In [2]:
import laspy
import numpy as np
import geopandas as gpd
import shapely
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# 1457 trees froim lidR

In [3]:
las = laspy.read("../data/teak_data/lidarAOI_g.las")

In [4]:
las_vstack = np.vstack([np.array(las.x), np.array(las.y), np.array(las.z)]).T

In [5]:
# get ground indices
ground_idx = np.where(np.array(las.classification) !=2)

In [6]:
len(ground_idx[0])

239339

In [7]:
len(np.array(las.classification)[ground_idx])

239339

In [8]:
las_vstack_veg = las_vstack[ground_idx[0]]

In [9]:
len(las_vstack_veg)

239339

In [6]:
%matplotlib qt

In [38]:
# plot original las data
fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(las_vstack_veg[:,0], las_vstack_veg[:,1], las_vstack_veg[:,2], s=12)
ax.view_init(azim=200)
plt.show()


/tmp/ipykernel_140957/591299872.py:2: MatplotlibDeprecationWarning: Axes3D(fig) adding itself to the figure is deprecated since 3.4. Pass the keyword argument auto_add_to_figure=False and use fig.add_axes(ax) to suppress this warning. The default value of auto_add_to_figure will change to False in mpl3.5 and True values will no longer work in 3.6.  This is consistent with other Axes classes.
  ax = Axes3D(fig)


In [ ]:
 # DBSCAN

In [23]:
from sklearn.cluster import DBSCAN
model = DBSCAN(eps=20, min_samples=2)
model.fit_predict(las_vstack_veg)
pred = model.fit_predict(las_vstack_veg)



In [24]:
# eps of 2.5 (defualt params) min samples 2
# number of cluster found: 1027



# eps of 20, min samples of 50
print("number of cluster found: {}".format(len(set(model.labels_))))


number of cluster found: 1


In [47]:

print("number of cluster found: {}".format(len(set(model.labels_))))

number of cluster found: 1027


In [59]:
import matplotlib.cm as cm

fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(las_vstack_veg[:,0], las_vstack_veg[:,1], las_vstack_veg[:,2],c=model.labels_,
           cmap=cm.hsv, s=8)
ax.view_init(azim=200)
plt.show()

/tmp/ipykernel_140957/1102482420.py:4: MatplotlibDeprecationWarning: Axes3D(fig) adding itself to the figure is deprecated since 3.4. Pass the keyword argument auto_add_to_figure=False and use fig.add_axes(ax) to suppress this warning. The default value of auto_add_to_figure will change to False in mpl3.5 and True values will no longer work in 3.6.  This is consistent with other Axes classes.
  ax = Axes3D(fig)


In [85]:
# 

In [ ]:
from sklearn.cluster import AgglomerativeClustering
model = AgglomerativeClustering(n_clusters=None, compute_full_tree = True,
                                distance_threshold=10, linkage="ward")
# model.fit_predict(las_vstack_veg)
pred = model.fit_predict(las_vstack_veg)

In [ ]:
print("number of cluster found: {}".format(len(set(model.labels_))))

In [ ]:
## CREATING AND SAVING LAS DATA

In [ ]:
# save segmented data to las file
# 1. Create a new header
header = laspy.LasHeader(point_format=3, version="1.3")
header.add_extra_dim(laspy.ExtraBytesParams(name="segs", type=np.int32))
header.offsets = np.min(las_vstack_veg, axis=0)
header.scales = np.array([0.01, 0.01, 0.01])

# 2. Create a Las
seg_las = laspy.LasData(header)

seg_las.x = las_vstack_veg[:, 0]
seg_las.y = las_vstack_veg[:, 1]
seg_las.z = las_vstack_veg[:, 2]
seg_las.classification = np.array(las.classification)[ground_idx]
seg_las.user_data = model.labels_
seg_las.write("treeseg_teak.las")